# Titanic - Machine Learning from Disaster


Kaggle link: https://www.kaggle.com/c/titanic

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Import all the needed library and init Weights and Biases

In [2]:
import numpy as np

import torch
import torch.nn as nn
torch.manual_seed(0)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import pandas as pd

import wandb
wandb.init(project='titanic_kaggle')

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


We first need to read the datasets

In [3]:
titanic_training_data = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
titanic_training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Dataframe needs to be cleaned, knowing if some informations are unknown can be very important to determine if someone survived

In [4]:
def clean_titanic(df, train=True):
    df["Cabin"] = df["Cabin"].apply(lambda x: pd.isna(x)).astype(bool)
    df["Embarked"] = df["Embarked"].apply(lambda x: pd.isna(x)).astype(bool)
    df["AgeNan"] = df["Age"].apply(lambda x: pd.isna(x)).astype(bool)
    df = pd.concat([df, pd.get_dummies(df['Sex'], dtype='bool', prefix='sex_'), pd.get_dummies(df['Pclass'], dtype='bool', prefix='pclass_')], axis=1)
    df = df.drop(['PassengerId', 'Name','Ticket','Sex','Pclass'], axis=1)
    if train:
        df = df.drop(['Survived'], axis=1)
    numeric_features = df.dtypes[(df.dtypes != 'object') & (df.dtypes != 'bool')].index
    df[numeric_features] = df[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
    df["Age"] = df["Age"].fillna(df["Age"].mean())
    df["Fare"] = df["Fare"].fillna(df["Fare"].mean())
    return df

labels = torch.tensor(titanic_training_data["Survived"].values, dtype=torch.float32)
titanic_training_data = clean_titanic(titanic_training_data)
titanic_training_data.head()

,Age,SibSp,Parch,Fare,Cabin,Embarked,AgeNan,sex__female,sex__male,pclass__1,pclass__2,pclass__3
0,-0.530005,0.432550,-0.473408,-0.502163,True,False,False,False,True,False,False,True
1,0.571430,0.432550,-0.473408,0.786404,False,False,False,True,False,True,False,False
2,-0.254646,-0.474279,-0.473408,-0.488580,True,False,False,True,False,False,False,True
3,0.364911,0.432550,-0.473408,0.420494,False,False,False,True,False,True,False,False
4,0.364911,-0.474279,-0.473408,-0.486064,True,False,False,False,True,False,False,True


We then transform the data from numpy (pandas representation) into torch's `Tensor`

In [5]:
titanic_data_tensor = torch.tensor(titanic_training_data.astype('float').values, dtype=torch.float32)
labels.shape

torch.Size([891])

Create a `TensorDataset` to get tuple of data and label

In [6]:
dataset = torch.utils.data.TensorDataset(titanic_data_tensor, labels)

We then split between the training and validation set

In [7]:
training_size = int(0.7 * len(dataset))
validation_size = len(dataset) - training_size
train, val = torch.utils.data.random_split(dataset, [training_size, validation_size], generator=torch.Generator().manual_seed(0))
data_loader_train = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
data_loader_val = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)

Layer initialization using Xavier Uniform on the weight and a constant 0 value on the bias

In [8]:
def weight_init(m):
    if isinstance(m,nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
        return m
    

Create the LinearModel with one Linear layer and Sigmoid applied to the output

In [9]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.ln = weight_init(nn.Linear(12,1))
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.ln(x)
        return self.Sigmoid(x)

Initialize the network (call it `net`, it would makes things easier later), the loss, the optimizer and write the training loop

Don't forget to check the validation loss and save your model at the end of each epoch!

In [10]:
num_epochs = 400 # should be more than enought, but can be changed
lr = 0.01 # e.q to 0.003, you can change it if needed

net = LinearModel()

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
min_val = np.inf

for epoch in range(num_epochs):
    training_loss = 0

    for x,target in data_loader_train:
        optimizer.zero_grad()
        net.train()
        prediction = net.forward(x)
        prediction = prediction.reshape(1,-1)
        target = target.reshape(1,-1)
        loss = criterion(prediction,target)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    training_loss /= len(data_loader_train)
    validation_loss = 0
    for x,target in data_loader_val:
        net.eval()
        with torch.no_grad():
            prediction_val = net.forward(x)
            prediction_val = prediction_val.reshape(1,-1)
            target = target.reshape(1,-1)
            loss = criterion(prediction_val,target)
            validation_loss += loss.item()
    validation_loss /= len(data_loader_val)
    if validation_loss > min_val:
        torch.save(net.state_dict(), 'saved_model.pth')
        
    wandb.log({'training_loss': training_loss, 'validation_loss': validation_loss})

print("Done")    

Done


This loop computes the prediction on the test dataset and create a submission file

You then just have to click the submit button to get your score, lucky you!

In [11]:
titanic_test_data_cleaned = clean_titanic(titanic_test_data, train=False)
titanic_data_tensor = torch.tensor(titanic_test_data_cleaned.astype('float').values, dtype=torch.float32)

with torch.no_grad():
    net.eval()
    test_pred = torch.LongTensor()
    for i, data in enumerate(titanic_data_tensor):
        output = net(data)
        predicted = torch.ge(output, 0.5)
        test_pred = torch.cat((test_pred, predicted), dim=0)
    out_df = pd.DataFrame(np.c_[titanic_test_data['PassengerId'].values, test_pred.numpy()], columns=['PassengerId', 'Survived'])
    out_df.to_csv('submission.csv', index=False)
out_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
